In [78]:
import requests
import pandas as pd 
import time
from bs4 import BeautifulSoup

origURL = 'https://www.myshiptracking.com/ports-arrivals-departures/?mmsi=&pid=89&type=0&time=1573820700_1573907100&page='

data = []
i = 1
while(i <= 50):
    url = origURL + '%d' % i

    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    rows = soup.select('div.cs-table div.table-group')
    for row in rows:
        data.append(row.select_one('div.table-row').text.strip().splitlines())
    i+= 1
    time.sleep(0)
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['Event', 'Time', 'Port', 'Vessel']) 
# print dataframe. 

In [79]:
print(df) 

         Event              Time       Port                  Vessel
0    Departure  2019-11-16 12:24  AMSTERDAM       MAJESTIC TMS [NL]
1      Arrival  2019-11-16 12:24  AMSTERDAM          MANTYRANO [NL]
2      Arrival  2019-11-16 12:23  AMSTERDAM               WABO [NL]
3    Departure  2019-11-16 12:23  AMSTERDAM             PONT 6 [NL]
4    Departure  2019-11-16 12:22  AMSTERDAM       VOLHARDING 8 [NL]
5    Departure  2019-11-16 12:21  AMSTERDAM            ANTARES [NL]
6    Departure  2019-11-16 12:21  AMSTERDAM               FREJ [NL]
7    Departure  2019-11-16 12:21  AMSTERDAM        STANLEYSTAD [NL]
8      Arrival  2019-11-16 12:18  AMSTERDAM             ADANDI [NL]
9      Arrival  2019-11-16 12:16  AMSTERDAM               ANNA [NL]
10   Departure  2019-11-16 12:16  AMSTERDAM  ATLANTIC PROGRESS [NL]
11     Arrival  2019-11-16 12:16  AMSTERDAM             PONT 6 [NL]
12     Arrival  2019-11-16 12:15  AMSTERDAM   JOHANNES VERMEER [NL]
13     Arrival  2019-11-16 12:11  AMSTERDAM     